# Scenario 4 model built with MIRA

In [ ]:
import sympy
import numpy
import matplotlib.pyplot as plt

from mira.metamodel import *
from mira.modeling import Model
from mira.modeling.viz import GraphicalModel
from mira.modeling.ode import OdeModel, simulate_ode_model
from mira.modeling.amr.regnet import template_model_to_regnet_json
from mira.modeling.amr.petrinet import template_model_to_petrinet_json


# Step 1: start with a minimal model of microbial growth

In [ ]:
species = lambda: Concept(name='x')
intrinsic = NaturalReplication(subject=species())
intrinsic.set_mass_action_rate_law('r')
interaction = ControlledReplication(controller=species(),
                                    subject=species())
interaction.set_mass_action_rate_law('a')
initial = Initial(concept=species(), expression=sympy.Float(1))
params = {'r': Parameter(name='r', value=1), 'a': Parameter(name='a', value=1)}

In [ ]:
tm = TemplateModel(templates=[intrinsic, interaction],
                   initials={'x': initial},
                   parameters=params)

In [ ]:
GraphicalModel.for_jupyter(tm)

## Step 2: stratify the minimal model into 4 and then 6 species

We capture model parameters and initial conditions here for all 6 species (the 4 species case is a subset of it).

In [ ]:
r_i = [0.53, 0.42, 0.49, 0.33, 0.7, 0.3]
x0_i = [0.51, 0.39, 0.88, 0.4, 0.2, 0.8]
a_ij = [
    [  -0.5, - 0.01, 0.002, -0.009, -0.002,  0.01],
    [     0,   -0.5,     0, -0.169,      0,     0],
    [-0.002, -0.003,  -0.5,   0.02,   0.03, -0.04],
    [     0, -0.226, -0.04,   -0.5,      0,  0.01],
    [     0,   -0.1, -0.02,      0,   -0.5,     0],
    [     0,  -0.04, -0.05,      0,      0,  -0.5]
]

In [ ]:
# Stratify into 4 species
tm_strat4 = stratify(tm, key='species',
                     strata=[str(i) for i in range(1, 5)],
                     structure=[],
                     cartesian_control=True)

# Stratify into 6 species
tm_strat6 = stratify(tm, key='species',
                     strata=[str(i) for i in range(1, 7)],
                     structure=[],
                     cartesian_control=True)

Model structure for 4-species model

In [ ]:
GraphicalModel.for_jupyter(tm_strat4)

Model structure for 6-species model

In [ ]:
GraphicalModel.for_jupyter(tm_strat6)

Reparameterize stratified models based on provided parameters

In [ ]:
# This code takes care of correctly reparameterizing the model
# after stratification
for tm_strat in [tm_strat4, tm_strat6]:
    for idx, init in enumerate(tm_strat.initials.values()):
        init.expression = SympyExprStr(sympy.Float(x0_i[idx]))
    for template in tm_strat.templates:
        ints = [int(i.context['species']) for i in template.get_interactors()]
        param = list(template.get_parameter_names())[0]
        if param.startswith('r'):
            tm_strat.parameters[param].value = r_i[ints[0]-1]
        else:
            tm_strat.parameters[param].value = a_ij[ints[1]-1][ints[0]-1]      

## Step 3: assemble models into ODEs and simulate
### Step 3.1: 4-species system

In [ ]:
om = OdeModel(Model(tm_strat4), initialized=True)

These are the RHS equations of the 4-species ODE system without substitution

In [ ]:
om.kinetics

In [ ]:
ts = numpy.linspace(0, 30, 100)
res = simulate_ode_model(om, times=ts)

These are the RHS equations of the 4-species ODE system with substitution

In [ ]:
om.kinetics

Simulation results for the 4-species system

In [ ]:
plt.plot(ts, res, label=range(1,5))
plt.ylim([0, 1.2])
plt.grid(alpha=0.25)
plt.legend(loc="center left", bbox_to_anchor=(1, 0.5))
plt.xlabel('Time')
plt.ylabel('Proportion')

### Step 3.2: 6-species system

In [ ]:
om = OdeModel(Model(tm_strat6), initialized=True)

In [ ]:
res = simulate_ode_model(om, times=numpy.linspace(0, 30, 100))

In [ ]:
om.kinetics

In [ ]:
plt.plot(ts, res, label=range(1,7))
plt.ylim([0, 1.25])
plt.grid(alpha=0.25)
plt.legend(loc="center left", bbox_to_anchor=(1, 0.5))
plt.xlabel('Time')
plt.ylabel('Proportion')

## Part 4. Export models into exchange formats

In [ ]:
frameworks = {'regnet': template_model_to_regnet_json, 'petrinet': template_model_to_petrinet_json}
models = {'4spec': tm_strat4, '6spec': tm_strat6}

for mkey, model in models.items():
    for fkey, fun in frameworks.items():
        with open(f'scenario4_{mkey}_{fkey}.json', 'w') as fh:
            json.dump(fun(model), fh, indent=1)